# Setup

Import pandas as usual, but also import numpy. We'll need it for logarithms and exponents.

Some of our datasets have a lot of columns, so you'll also want to use `pd.set_option` to display up to 100 columns or so.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
pd.set_option("display.max_columns", 100)
# pd.set_option("display.max_colwidth", 100)
# pd.reset_option('display.max_columns')

# What is each row of our data?

If you aren't sure, you might need to look at either the whitepaper or the codebook. You'll need to look at them both eventually, so might as well get started now.

# Lending disparities using Logistic Regression

**The story:** https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/

**Author:** Aaron Glantz and Emmanuel Martinez

**Topics:** Logistic regression, odds ratios

**Datasets**

* **philadelphia-mortgages.csv:** Philadelphia mortgage data for 2015
    - A subset of HMDA LAR data from [FFEIC](https://www.ffiec.gov/hmda/hmdaproducts.htm)
    - Codebook is `2015HMDACodeSheet.pdf`
    - A [guide to HMDA reporting](https://www.ffiec.gov/hmda/guide.htm)
    - I've massaged it slightly to make processing a bit easier
* **nhgis0006_ds233_20175_2017_tract.csv:**
    - Table B03002: Hispanic or Latino Origin by Race
    - 2013-2017 American Community Survey data US Census Bureau, from [NHGIS](https://data2.nhgis.org/main)
    - Codebook is `nhgis0006_ds233_20175_2017_tract_codebook.txt`
* **lending_disparities_whitepaper_180214.pdf:** the whitepaper outlining Reveal's methodology

## What's the goal?

Do banks provide mortgages at disparate rates between white applicants and people of color? We're going to look at the following variables to find out:

* Race/Ethnicity
    - Native American
    - Asian
    - Black
    - Native Hawaiian
    - Hispanic/Latino
    - Race and ethnicity were not reported
* Sex
* Whether there was a co-applicant
* Applicant’s annual income (includes co-applicant income)
* Loan amount
* Ratio between the loan amount and the applicant’s income
* Ratio between the median income of the census tract and the median income of the metro area
* Racial and ethnic breakdown by percentage for each census tract
* Regulating agency of the lending institution

# Read in your data

Read in our Philadelphia mortgage data and take a peek at the first few rows.

* **Tip:** As always, census tract columns like to cause problems if they're read in as numbers. Make sure pandas reads it in as a string.

In [3]:
df = pd.read_csv('sources/philadelphia-mortgages.csv', dtype='str')
df.census_tract = df.census_tract.str.replace('.','')
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex
0,010100,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5
1,026400,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5
2,028100,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5
3,015800,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5
4,035800,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2


### Check your column types

I mentioned it above, but make sure `census_tract` is an object (a string) or merging isn't going to be any fun later on.

In [4]:
df.dtypes

census_tract                   object
county_code                    object
state_code                     object
applicant_sex                  object
income                         object
loan_amount                    object
loan_type                      object
property_type                  object
occupancy                      object
action_type                    object
loan_purpose                   object
agency_code                    object
tract_to_msa_income_percent    object
applicant_race                 object
co_applicant_sex               object
dtype: object

# Engineering and cleaning up features

## Income-related columns

> When we plotted the number of applicants, how much money they made and the size of the loan, we found that it skewed to the left, meaning the majority of applicants were clustered on the lower end of the income and loan amount scales. This was especially true for applicants of color. **We took the logarithm transformation of income and loan amount to normalize the distribution of those variables and limit the effect of extreme outliers.**

A few of the columns you'll need to calculate yourselves. **Calculate these values and assign them to three new columns.**

* Applicant’s adjusted annual income (includes co-applicant income)
* Adjusted loan amount
* Ratio between the loan amount and the applicant’s income

Instead of using the raw income and loan amount, you'll want the log of both income and loan amount. Call these new columns `log_income` and `log_loan_amount`. The third column will be `loan_income_ratio`.

* **Tip:** `np.log` gives you the logarithm

In [5]:
df['log_income'] = np.log(df.income.astype(int)*1000).round(2)
df['log_loan_amount'] = np.log(df.loan_amount.astype(int)*1000).round(2)
df['loan_income_ratio'] = (df.log_loan_amount / df.log_income).round(4)
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio
0,010100,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5,10.17,8.52,0.8378
1,026400,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5,10.17,10.60,1.0423
2,028100,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5,10.00,9.90,0.9900
3,015800,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5,10.95,10.49,0.9580
4,035800,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2,11.29,10.43,0.9238


### Co-applicants

Right now we have a column about the co-applicant's sex (see the codebook for column details). We don't want the sex, though, we're interested in whether there is a co applicant or not. Use the co-applicant's sex to **create a new column called `co_applicant` that is either 'yes', 'no', or 'unknown'.**

* **Hint:** If the co-applicant's sex was not provided or is not applicable, count it as unknown.
* **Hint:** The easiest way is to use `.replace` on the co-applicant sex column, but store the result in your new column

In [6]:
df.co_applicant_sex.value_counts()

5    40154
2    13014
1     4968
3     1529
4     1040
Name: co_applicant_sex, dtype: int64

In [7]:
df['co_applicant'] = df.co_applicant_sex.replace({'1': 'Yes', '2': 'Yes', '3': 'Unknown', '4': 'Unknown', '5': 'No'})

In [8]:
df.co_applicant.value_counts()

No         40154
Yes        17982
Unknown     2569
Name: co_applicant, dtype: int64

# Filter loan applicants

If you read the whitepaper - `lending_disparities_whitepaper_180214.pdf` - many filters are used to get to the target dataset for analysis.

> **Loan type**
>
> While we recognize the substantial presence of applicants of color in the FHA market, we focused on conventional home loans for several reasons.

> **Property type**
>
> Prospective borrowers submit loan applications for various types of structures: one- to four-unit properties, multifamily properties and manufactured homes. For this analysis, we focused on one- to four-unit properties.

> **Occupancy**
>
> We included only borrowers who said they planned to live in the house they were looking to buy. We did this to exclude developers or individuals who were buying property as an investment or to subsequently flip it.

> **Action Type**
>
> We wanted to look at the reasons lending institutions deny people a mortgage. After conversations with former officials at HUD, we decided to include only those applications that resulted in originations (action type 1) or denials (action type 3)

> **Income**
>
> An applicant’s income isn’t always reported in the data. In other cases, the data cuts off any incomes over \\$9.9 million and any loan amounts over \\$99.9 million, meaning there’s a value in the database, but it’s not precise. We focused only on those records where income and loan amount have an accurate estimation. This meant discarding about 1 percent of all conventional home loans in the country for 2016. **[Note: I already edited this]**
>
> When we plotted the number of applicants, how much money they made and the size of the loan, we found that it skewed to the left, meaning the majority of applicants were clustered on the lower end of the income and loan amount scales. This was especially true for applicants of color. We took the logarithm transformation of income and loan amount to normalize the distribution of those variables and limit the effect of extreme outliers.

> **Lien status**
>
> We included all cases in our analysis regardless of lien status.

> **Race and ethnicity**
>
> At first, we looked at race separate from ethnicity, but that approach introduced too many instances in which​ ​either the ethnicity or race was unknown. So we decided to combine race and ethnicity. Applicants who marked their ethnicity as Hispanic were grouped together as Hispanic/Latino regardless of race. Non-Hispanic applicants, as well as those who didn’t provide an ethnicity, were grouped together by race: non-Hispanic white, non-Hispanic black, etc. **[Note: This has already been taken care of]**

> **Loan purpose**
>
> We decided to look at home purchase, home improvement and refinance loans separately from each other. [Note: please look at **home purchase** loans.]

Use the text above (it's from the whitepaper) and the **2015HMDACodeSheet.pdf** code book to filter the dataset.

* **Tip:** there should be between 5-8 filters, depending on how you write them.

In [9]:
# Loan type
df = df[df.loan_type.str.contains('1')]
# Property type
df = df[df.property_type.str.contains('1')]
# Occupancy
df = df[df.occupancy.str.contains('1')]
# Action Type
df = df[df.action_type.str.contains('1|3')]
# Loan purpose
purchase_df = df[df.loan_purpose.str.contains('1')]
improve_df = df[df.loan_purpose.str.contains('2')]
refinance_df = df[df.loan_purpose.str.contains('3')]

When you're done filtering, save your dataframe as a "copy" with `df = df.copy()` (if it's called `df`, of course). This will prevent irritating warnings when you're trying to create new columns.

In [10]:
purchase_df = purchase_df.copy()
# deleting now unused columns
purchase_df = purchase_df.drop(['loan_type','property_type','occupancy','loan_purpose'], axis=1)

### Confirm that you have 10,107 loans with 19 columns

In [11]:
purchase_df.shape

(10107, 15)

### Create a "loan denied" column

Right now the `action_type` category reflects whether the loan was granted or not, and either has a value of `1` or `3`.

Create a new column called `loan_denied`, where the value is `0` if the loan was accepted and `1` if the loan was denied. **This will be our target for the machine learning algorithm.**

* **Tip:** You should have 8,878 successful loans and 1,229 denied loans

In [12]:
purchase_df['loan_denied'] = purchase_df.action_type.replace({'1':'0', '3':'1'}).astype(int)
purchase_df.loan_denied.value_counts()

0    8878
1    1229
Name: loan_denied, dtype: int64

In [13]:
purchase_df.dtypes

census_tract                    object
county_code                     object
state_code                      object
applicant_sex                   object
income                          object
loan_amount                     object
action_type                     object
agency_code                     object
tract_to_msa_income_percent     object
applicant_race                  object
co_applicant_sex                object
log_income                     float64
log_loan_amount                float64
loan_income_ratio              float64
co_applicant                    object
loan_denied                      int32
dtype: object

# Deal with categorical variables

Let's go ahead and take a look at our categorical variables:

* Applicant sex (male, female, na)
* Applicant race
* Mortgage agency
* Co-applicant (yes, no, unknown)

Before we do anything crazy, let's use the codebook to turn them into strings.

* **Tip:** We already did this with the `co_applicant` column, you only need to do the rest
* **Tip:** Just use `.replace`

In [14]:
purchase_df.applicant_sex.replace({'1':'Male', '2':'Female', '3':'unknown'}, inplace=True)
purchase_df.applicant_race.replace({'1':'NA', '2':'Asian', '3':'Black', '4':'PI', '5':'White', '6':'Unknown', '99':'Hisp'}, inplace=True)
purchase_df.co_applicant_sex.replace({'1':'Male', '2':'Female', '3':'Unknown', '4':'Unknown', '5':'None'}, inplace=True)
purchase_df.agency_code.replace({'1':'OCC', '2':'FRS', '3':'FDIC', '5':'NCUA', '7':'HUD', '9':'CFPB'}, inplace=True)

In [15]:
purchase_df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,action_type,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio,co_applicant,loan_denied
42,401900,045,42,Female,59,112,1,OCC,133.09,White,None,10.99,11.63,1.0582,No,0
43,409902,045,42,unknown,177,375,1,OCC,208.56,Unknown,Unknown,12.08,12.83,1.0621,Unknown,0
46,410200,045,42,Male,150,381,1,OCC,215.35,White,None,11.92,12.85,1.0780,No,0
48,031200,101,42,Female,65,136,1,OCC,93.11,Asian,None,11.08,11.82,1.0668,No,0
51,403601,045,42,Female,55,196,1,OCC,141.83,Asian,None,10.92,12.19,1.1163,No,0


Double-check these columns match these values in the first three rows (and yes, you should have a lot of other columns, too).

|applicant_sex|agency_code|applicant_race|co_applicant|
|---|---|---|---|
|female|OCC|white|no|
|na|OCC|na|unknown|
|male|OCC|white|no|

## Dummy variables

Let's say we're at the end of the homework, and we have a column called `sex`, where `0` is female and `1` is male. After we've done our regression, we can look at the coefficient/odds ratio for `sex` and say something like **"being male gives you a 1.5x odds of being denied a loan."**

We can say this because we're looking at one column, and changing `sex` from `0` to `1` would turn the applicant male and give them a 1.5x chance of being denied (the odds ratio).

**But let's say we're looking at a column called `race` instead.** We could do the same `0`/`1` thing with white/minority, but what about white/black/asian? If we try to give them `0`/`1`/`2` our coefficient/odds ratio interpreation stops working, because we don't have a nice True/False dichotomy any more, it's now a *real number*.

* `0`: White
* `1`: Black
* `2`: Asian

Usually with numbers you can say "...for every increase of 1...", but we can't anymore - changing from White to Black (+1) isn't the same as changing from Black to Asian (+1). And you can't subtract Black from Asian to get White. And no, you also can't average together White and Asian to get Black. Just recognize that these aren't numbers, they're categories!

**How can we turn races off and on like we can turn the `sex` variable off and on?** A good option is to make *a `0`/`1` column for each race*. We can then flip each race off and on. These are called **dummy variables**.

In [16]:
pd.get_dummies(purchase_df.applicant_race, prefix='race').head()

,race_Asian,race_Black,race_Hisp,race_NA,race_PI,race_Unknown,race_White
42,0,0,0,0,0,0,1
43,0,0,0,0,0,1,0
46,0,0,0,0,0,0,1
48,1,0,0,0,0,0,0
51,1,0,0,0,0,0,0


Seems to take up a lot of space, but it works a lot better.

* The first person is white, so they have a `1` for white and a `0` for every other race
* The second person is N/A, so they have a `1` for N/A and a `0` for every other race
* The next three are white, asian, and asian, so they have a `1` under the appropriate column.

When you're looking at the regression output, each column has its own coefficient (and odds ratio). Since each race now has a column, **each race will also have its own odds ratio.** Asian would have one, Black would have one, Latino would have one - now we can look at the effect of each race separately. For example, you could then say something like "being Asian (e.g., `race_asian` going from `0` to `1`) gives you a 1.2x greater chance of being denied, and being Black gets you a 2.1x chance of being denied."

And no, you're never going to have more than one `1` in a row at the same time.

After you've created your dummy variables, there's one more step which has a real fun name: **one-hot encoding.**

### One-hot encoding

When we have two sexes - male and female - we can flip between them with one binary digit, `0` and `1`.

If we had three races - White, Asian, Black - using `pd.get_dummies` would make three columns, which makes sense on the surface. But why can we put TWO values in ONE column for sex, and it takes THREE columns for the THREE race values?

The truth is, it doesn't have to!

Instead of having three columns, we're only going to have two: **asian and black**. And if both of them are `0`? The applicant is white! This is called a **reference category**, and it means **the coefficients/odds ratios for asian and black are in reference to a white applicant.** So it isn't "being black gets you a 2.1x chance of being denied," it's *being black gets you a 2.1x chance of being denied compared to a white person*. For example:

|race_asian|race_black|person's race|
|---|---|---|
|1|0|Asian|
|0|1|Black|
|0|0|White|
|1|1|Not possible if your source is a single race column|

To create a one-hot encoded variable with a reference category, you write code like this:

In [17]:
pd.get_dummies(purchase_df.applicant_race, prefix='race').drop('race_White', axis=1).head()

,race_Asian,race_Black,race_Hisp,race_NA,race_PI,race_Unknown
42,0,0,0,0,0,0
43,0,0,0,0,0,1
46,0,0,0,0,0,0
48,1,0,0,0,0,0
51,1,0,0,0,0,0


> We usually use `.drop(columns=...)` to drop columns, but I'm using `axis=1` here because you should be familiar with it

### Make a one-hot encoded `sex` category with `female` as the reference category

You should end up with two columns: `sex_male` and `sex_na`.

In [18]:
pd.get_dummies(purchase_df.applicant_sex, prefix='sex').drop('sex_Female', axis=1).head()

,sex_Male,sex_unknown
42,0,0
43,0,1
46,1,0
48,0,0
51,0,0


## Using one-hot encoded columns

Since these one-hot encoded variables are standalone dataframes, we eventually need to combine them into our original dataframe.

We have four categorical variables - sex, race, co-applicant, and the loan agency - so we need you to **make four one-hot encoded variables**. Name them like this:

* `dummies_sex` - reference category of white
* `dummies_race` - reference category of female
* `dummies_co_applicant` - reference category of no
* `dummies_agency` - reference category of FDIC

Typically your reference category is the most common category, because it makes for the most interesting comparisons.

> **Tip:** if you're cutting and pasting from above, watch out for `.head()`
>
> **Tip:** After you've made them, use `.head(2)` to check the first couple rows of each to make sure they look okay

In [19]:
dummies_race = pd.get_dummies(purchase_df.applicant_race, prefix='race').drop('race_White', axis=1)
dummies_sex = pd.get_dummies(purchase_df.applicant_sex, prefix='sex').drop('sex_Female', axis=1)
dummies_co_applicant = pd.get_dummies(purchase_df.co_applicant, prefix='co_app').drop('co_app_No', axis=1)
dummies_agency = pd.get_dummies(purchase_df.agency_code, prefix='agency').drop('agency_FDIC', axis=1)

## Cleaning up our old dataframe

Take a look at your original dataframe real quick.

In [20]:
purchase_df.head(2)

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,action_type,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio,co_applicant,loan_denied
42,401900,045,42,Female,59,112,1,OCC,133.09,White,None,10.99,11.63,1.0582,No,0
43,409902,045,42,unknown,177,375,1,OCC,208.56,Unknown,Unknown,12.08,12.83,1.0621,Unknown,0


We don't need all of those columns! If we look at the list of columns we'll be using for the regression:

* Race/Ethnicity
* Sex
* Whether there was a co-applicant
* Applicant’s annual income (includes co-applicant income)
* Loan amount
* Ratio between the loan amount and the applicant’s income
* Ratio between the median income of the census tract and the median income of the metro area
* Racial and ethnic breakdown by percentage for each census tract
* Regulating agency of the lending institution

We can keep anything in that list, and remove everything else. For example, we can drop the variables we used to create the dummy variables, as we'll be adding the one-hot encoded versions in for the next step.

For "Racial and ethnic breakdown by percentage for each census tract" we'll need to join with some census data later, so we need to also keep census tract, county code and state code.

**Build a new dataframe with only the columns we're interested in, call it `numeric`.** We're calling it `numeric` because it's mostly numeric columns after the categorical ones have been removed.

> **Tip:** You can either use `.drop(columns=` to remove unwanted columns or `df = df[['col1', 'col2', ... 'col12']]` to only select the ones you're interseted in

In [21]:
numeric = purchase_df[['census_tract','county_code','state_code','tract_to_msa_income_percent','log_income','log_loan_amount','loan_income_ratio','loan_denied']]

Confirm that `numeric` has 8 columns.

In [22]:
numeric.head()

,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied
42,401900,045,42,133.09,10.99,11.63,1.0582,0
43,409902,045,42,208.56,12.08,12.83,1.0621,0
46,410200,045,42,215.35,11.92,12.85,1.0780,0
48,031200,101,42,93.11,11.08,11.82,1.0668,0
51,403601,045,42,141.83,10.92,12.19,1.1163,0


### Combining our features

We now have 1 dataframe of numeric features (and some merge columns), and 4 one-hot-encoded variables (each with their own dataframe). Combine all five dataframes into one large dataframe called `loan_features`.

In [23]:
loan_features = pd.concat([numeric, dummies_race, dummies_sex, dummies_co_applicant, dummies_agency], axis=1)
loan_features.head()

,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied,race_Asian,race_Black,race_Hisp,race_NA,race_PI,race_Unknown,sex_Male,sex_unknown,co_app_Unknown,co_app_Yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC
42,401900,045,42,133.09,10.99,11.63,1.0582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
43,409902,045,42,208.56,12.08,12.83,1.0621,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1
46,410200,045,42,215.35,11.92,12.85,1.0780,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
48,031200,101,42,93.11,11.08,11.82,1.0668,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
51,403601,045,42,141.83,10.92,12.19,1.1163,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


Confirm that `loan_features` has 10,107 rows and 23 columns.

In [24]:
loan_features.shape

(10107, 23)

# Census data

Now we just need the final piece to the puzzle, the census data. Read in the census data file, calling the dataframe `census`.

> **Tip:** As always, be sure to read the tract column in as a string. Interestingly, this time we _don't_ need to worry about the state or county codes in the same way.
>
> **Tip:** You're going to encounter a problem that you find every time you read in a file from the US government!

In [25]:
columns = ['STATEA',
           'COUNTYA',
           'TRACTA',
           'ADK5E001',
           'ADK5E002','ADK5E003','ADK5E004','ADK5E005','ADK5E006','ADK5E007','ADK5E008','ADK5E009','ADK5E010','ADK5E011',
           'ADK5E012','ADK5E013','ADK5E014','ADK5E015','ADK5E016','ADK5E017','ADK5E018','ADK5E019','ADK5E020','ADK5E021']
census = pd.read_csv('sources/nhgis0007_ds215_20155_2015_tract.csv', encoding='latin-1', usecols=columns, dtype={'STATEA':str,'COUNTYA':str,'TRACTA':str})
census.rename(columns={'STATEA':'state_code',
                       'COUNTYA':'county_code',
                       'TRACTA':'census_tract',
                       'ADK5E001':'Total',
                       'ADK5E002':'NoHisp_All',
                       'ADK5E003':'NoHisp_White',
                       'ADK5E004':'NoHisp_Black',
                       'ADK5E005':'NoHisp_NA',
                       'ADK5E006':'NoHisp_Asian',
                       'ADK5E007':'NoHisp_PI',
                       'ADK5E008':'NoHisp_Other',
                       'ADK5E009':'NoHisp_Multi1',
                       'ADK5E010':'NoHisp_Multi2',
                       'ADK5E011':'NoHisp_Multi3',
                       'ADK5E012':'Hisp_All',
                       'ADK5E013':'Hisp_White',
                       'ADK5E014':'Hisp_Black',
                       'ADK5E015':'Hisp_NA',
                       'ADK5E016':'Hisp_Asian',
                       'ADK5E017':'Hisp_PI',
                       'ADK5E018':'Hisp_Other',
                       'ADK5E019':'Hisp_Multi1',
                       'ADK5E020':'Hisp_Multi2',
                       'ADK5E021':'Hisp_Multi3'
                      }, inplace=True)
census.sample(5)

,state_code,county_code,census_tract,Total,NoHisp_All,NoHisp_White,NoHisp_Black,NoHisp_NA,NoHisp_Asian,NoHisp_PI,NoHisp_Other,NoHisp_Multi1,NoHisp_Multi2,NoHisp_Multi3,Hisp_All,Hisp_White,Hisp_Black,Hisp_NA,Hisp_Asian,Hisp_PI,Hisp_Other,Hisp_Multi1,Hisp_Multi2,Hisp_Multi3
10329,06,081,608400,5568,4347,2380,0,0,1555,211,0,201,0,201,1221,788,0,0,69,0,284,80,0,80
72765,55,133,200202,4178,4156,3816,41,0,255,0,28,16,0,16,22,22,0,0,0,0,0,0,0,0
20678,16,001,001100,4261,3029,2886,19,27,4,0,0,93,0,93,1232,760,0,0,0,0,171,301,0,301
50901,39,035,103300,2468,2304,660,1548,0,51,0,0,45,0,45,164,23,77,0,0,0,44,20,0,20
65731,48,381,021703,6166,4896,4712,157,19,0,0,0,8,0,8,1270,1030,6,0,0,0,234,0,0,0


## Rename some columns

If you like to keep your data extra clean, feel free to rename the columns you're interested in. If not, feel free to skip it!

> **Tip:** Make sure you're using the estimates columns, not the margin of error columns

In [26]:
# Completed earlier

## Computed columns

According to Reveal's regression output, you'll want to create the following columns:

* Percent Black in tract
* Percent Hispanic/Latino in tract (I hope you know how Hispanic/Latino + census data works by now)
* Percent Asian in tract
* Percent Native American in tract
* Percent Native Hawaiian in tract

Notice that we don't include percent white - **because all of the other columns add up to percent white, we ignore it!** It's similar to a reference category.

> If we want to use buzzwords here, the technical reason we're not using percent white is called **collinearity.** We'll talk more about it on Friday.

In [27]:
census_features = census[['state_code','county_code','census_tract']]
census_features['pct_Hisp'] = ((census.Hisp_All / census.Total) * 100).round(1)
census_features['pct_Black'] = ((census.NoHisp_Black / census.Total) * 100).round(1)
census_features['pct_NA'] = ((census.NoHisp_NA / census.Total) * 100).round(1)
census_features['pct_Asian'] = ((census.NoHisp_Asian / census.Total) * 100).round(1)
census_features['pct_PI'] = ((census.NoHisp_PI / census.Total) * 100).round(1)
census_features.sample(5)

,state_code,county_code,census_tract,pct_Hisp,pct_Black,pct_NA,pct_Asian,pct_PI
38659,29,189,220300,2.1,10.5,0.1,0.6,0.0
65661,48,373,210101,6.6,1.7,0.0,0.0,0.0
20136,13,263,960100,0.0,55.9,0.0,0.3,0.0
35995,27,053,025301,4.9,2.9,0.2,1.3,0.0
54479,40,145,030510,6.4,1.2,5.5,0.7,0.0


## Only keep what we need to join and process

We're only interested in the percentage columns that we computed. Create a new dataframe called `census_features` that is only those columns along with the one we'll need for joining with the mortgage data.

> * **Tip:** Remember we saved state, county and tract codes when working on the loan data

In [28]:
# Completed earlier

Confirm that your first few rows look something like this:
    
|STATEA|COUNTYA|TRACTA|pct_hispanic|pct_black|pct_amer_indian|pct_asian|pct_pac_islander|
|---|---|---|---|---|---|---|---|
|1|1|020100|0.872690|7.700205|0.308008|0.616016|0.000000|
|1|1|020200|0.788497|53.293135|0.000000|2.319109|0.000000|
|1|1|020300|0.000000|18.564690|0.505391|1.381402|0.269542|
|1|1|020400|10.490617|3.662672|1.560027|0.000000|0.000000|
|1|1|020500|0.743287|24.844374|0.000000|3.827929|0.000000|

Your column headers might be different but your numbers should match.

# Merge datasets

Merge `loan_features` and `census_features` into a new dataframe called `merged`.

Unfortunately something is a little different between our `loan_features` and `census_features` census tract columns. You'll need to fix it before you can merge.

## Cleaning

In [29]:
# Completed earlier

## Do the merge

In [30]:
merged = pd.merge(loan_features, census_features, on=['state_code','county_code','census_tract'])

Confirm you have 10107 rows and 31 columns in the merged dataframe.

In [31]:
merged.shape

(10107, 28)

In [32]:
merged.sample(3)

,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied,race_Asian,race_Black,race_Hisp,race_NA,race_PI,race_Unknown,sex_Male,sex_unknown,co_app_Unknown,co_app_Yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC,pct_Hisp,pct_Black,pct_NA,pct_Asian,pct_PI
9596,402300,045,42,100.51,11.03,11.74,1.0644,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3.3,94.1,0.4,0.4,0.0
6733,409000,045,42,189.21,11.90,12.46,1.0471,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0.5,0.5,0.0,11.1,0.0
1066,408001,045,42,185.09,11.34,12.49,1.1014,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,4.4,0.0,0.0,7.0,0.0


# Our final dataframe

Drop all of the columns we merged on and save it as `train_df`.

In [33]:
train_df = merged.drop(['census_tract','county_code','state_code'], axis=1)

Confirm that `train_df` has 10107 rows and 25 columns.

In [34]:
train_df.shape

(10107, 25)

## Final cleanup

Because we can't have missing data before we run a regression, check the size of `train_df`, then drop any missing data and check the size again. **Confirm you don't lose any rows.**

In [35]:
train_df = train_df.dropna(axis=0, how='any')

In [36]:
train_df.shape

(10107, 25)

In [37]:
train_df.dtypes

tract_to_msa_income_percent     object
log_income                     float64
log_loan_amount                float64
loan_income_ratio              float64
loan_denied                      int32
race_Asian                       uint8
race_Black                       uint8
race_Hisp                        uint8
race_NA                          uint8
race_PI                          uint8
race_Unknown                     uint8
sex_Male                         uint8
sex_unknown                      uint8
co_app_Unknown                   uint8
co_app_Yes                       uint8
agency_CFPB                      uint8
agency_FRS                       uint8
agency_HUD                       uint8
agency_NCUA                      uint8
agency_OCC                       uint8
pct_Hisp                       float64
pct_Black                      float64
pct_NA                         float64
pct_Asian                      float64
pct_PI                         float64
dtype: object

# Performing our regression

## Try with statsmodels

First try to run a linear regression with statsmodels, because even though sometimes it complains and breaks, the output just looks *so nice*. Instead of `sm.OLS` we'll use `sm.Logit`.

In [38]:
X = train_df[['race_Black','race_Hisp','race_Asian','pct_Black','pct_Hisp','pct_Asian','sex_Male','log_income']]
X = sm.add_constant(X)
y = train_df['loan_denied']               

mod = sm.Logit(y,X)
res = mod.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.352024
         Iterations 7


C:\Program Files\Python37\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            loan_denied   No. Observations:                10107
Model:                          Logit   Df Residuals:                    10098
Method:                           MLE   Df Model:                            8
Date:                Wed, 24 Jul 2019   Pseudo R-squ.:                 0.04884
Time:                        15:40:08   Log-Likelihood:                -3557.9
converged:                       True   LL-Null:                       -3740.6
Covariance Type:            nonrobust   LLR p-value:                 4.734e-74
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7043      0.559      4.838      0.000       1.609       3.800
race_Black     0.7801      0.104      7.468      0.000       0.575       0.985
race_Hisp      0.2032      0.157      1.296      0.195      -0.104       0.511
race_Asian     0.2429      0.099      2.447      0.014       0.048       0.437
pct_Black      0.0069      0.001      5.199      0.000       0.004       0.009
pct_Hisp       0.0033      0.003      1.058      0.290      -0.003       0.009
pct_Asian      0.0087      0.004      2.097      0.036       0.001       0.017
sex_Male       0.0030      0.064      0.047      0.962      -0.122       0.128
log_income    -0.4473      0.048     -9.246      0.000      -0.542      -0.352
==============================================================================
"""

## Try again with sci-kit learn

But I'll be honest, I like sklearn a *lot lot lot* better. Using the coefficient to build a dataframe just seems so *nice*.


In [41]:
X = train_df.drop(columns='loan_denied')
y = train_df.loan_denied

clf = lm.LogisticRegression() 
clf.fit(X,y)

results = pd.DataFrame(np.exp(clf.coef_), columns=X.columns)
results

C:\Program Files\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,race_Asian,race_Black,race_Hisp,race_NA,race_PI,race_Unknown,sex_Male,sex_unknown,co_app_Unknown,co_app_Yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC,pct_Hisp,pct_Black,pct_NA,pct_Asian,pct_PI
0,1.001753,0.827473,0.690361,7.415705,1.435427,2.082528,1.35929,2.262871,2.511771,1.541899,1.129008,0.914497,1.452673,0.90981,2.939619,0.889601,1.096451,3.562995,1.344595,1.007551,1.00656,0.776403,1.01093,1.085529


> **Tip:** When you build your model, use `LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000)` - for if you don't increase `max_iter` (how long/hard it works) it'll complain it can't find an answer.

In [42]:
X = train_df.drop(columns='loan_denied')
y = train_df.loan_denied

clf = lm.LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000) 
clf.fit(X,y)

results = pd.DataFrame(np.exp(clf.coef_), columns=X.columns)
results

,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,race_Asian,race_Black,race_Hisp,race_NA,race_PI,race_Unknown,sex_Male,sex_unknown,co_app_Unknown,co_app_Yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC,pct_Hisp,pct_Black,pct_NA,pct_Asian,pct_PI
0,1.001828,0.82493,0.649942,9.857884,1.437749,2.102847,1.391338,2.345192,2.955732,1.567729,1.131838,0.905971,1.509787,0.934386,3.054553,0.894539,1.132135,3.664148,1.396756,1.006543,1.005945,0.785062,1.010418,1.096453


### Getting your coefficients and odds ratios

After you run your regression **using sklearn**, you can use code like the below to print out an ordered list of features, coefficients, and odds ratios.

```python
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)
```

In [43]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
3,loan_income_ratio,2.288272,9.857884
17,agency_NCUA,1.298596,3.664148
14,agency_CFPB,1.116633,3.054553
8,race_PI,1.083746,2.955732
7,race_NA,0.852367,2.345192
5,race_Black,0.743292,2.102847
9,race_Unknown,0.449628,1.567729
12,co_app_Unknown,0.411969,1.509787
4,race_Asian,0.363079,1.437749
18,agency_OCC,0.334152,1.396756


### Wait, what's the odds ratio again?

It's how much that variable affects the outcome **if all other variables stay the same.**

# Interpreting and thinking about the analysis

### Question 1

Our results aren't exactly the same as Reveal's, as I pulled a slightly different number of rows from the database and I'm not sure what exact dataset they used for census information. How are we feeling about this reproduction? **You might want check their 2015 results in the whitepaper.**

### Question 2

In the opening paragraph to the flagship piece, [Aaron and Emmanuel write](https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/):

> Fifty years after the federal Fair Housing Act banned racial discrimination in lending, African Americans and Latinos continue to be routinely denied conventional mortgage loans at rates far higher than their white counterparts.

If you look at the results, Hawaiians/Pacific Islanders (and maybe Native Americans) have an even higher odds ratio. **Why do they choose to talk about African Americans and Latinos instead?**

### Question 3

Write a sentence expressing the meaning of the **odds ratio** statistic for Black mortgage applicants. Find a line in [the Reveal piece](https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/) where they use the odds ratio.

### Question 4

Write a similar sentence about men.

### Question 5

Why did Aaron and Emmanuel choose to include the loan-to-income ratio statistic? **You might want to read the whitepaper.**

### Question 6

Credit score is a common reason why loans are denied. Why are credit scores not included in our analysis? **You might want to read the whitepaper.**

### Question 7

This data was just sitting out there for anyone to look at, they didn't even need to FOIA it. Why do you think this issue had not come up before Reveal's analysis?

### Question 8

As a result of this series, [a lot has happened](https://www.revealnews.org/blog/we-exposed-modern-day-redlining-in-61-cities-find-out-whats-happened-since/), although [recent changes don't look so good](https://www.revealnews.org/blog/cfpb-moves-to-limit-home-loan-data/). If you were reporting this story, what groups of people would you want to talk to in order to make sure you're getting the story right?

### Question 9

When they were consulting experts, Aaron and Emmanuel received a lot of conflicting accounts about whether they should include the "N/A" values for race (they ended up including it). If the experts disagreed about something like that, why do you think they went forward with their analysis?

### Question 10

What if we were working on this story, and our logistic regression or input dataset were flawed? What would be the repercussions?